# Explore urban networks and amenities as interactive web maps

Author: [Geoff Boeing](https://geoffboeing.com/)

Use OSMnx to download a street network, calculate a route between two points, and create a Leaflet web map using GeoPandas [explore](https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoDataFrame.explore.html). See the GeoPandas documentation for more style details.

  - [Documentation](https://osmnx.readthedocs.io/)
  - [Journal article and citation info](https://geoffboeing.com/publications/osmnx-paper/)
  - [Code repository](https://github.com/gboeing/osmnx)
  - [Examples gallery](https://github.com/gboeing/osmnx-examples)

In [ ]:
import networkx as nx
import osmnx as ox

ox.__version__

In [ ]:
# download a street network then solve a shortest-path route on it
weight = "length"
G = ox.graph_from_place("Piedmont, CA, USA", network_type="drive")
orig = list(G.nodes)[0]
dest = list(G.nodes)[-1]
route = ox.shortest_path(G, orig, dest, weight=weight)

## Explore a city's street network as an interactive web map

In [ ]:
# explore graph edges interactively, with a simple one-liner
ox.graph_to_gdfs(G, nodes=False).explore()

You can zoom into the street network above or hover over any edge to get more info.

In [ ]:
# explore graph nodes interactively, with different basemap tiles
nodes = ox.graph_to_gdfs(G, edges=False)
nodes.explore(tiles="cartodbpositron", marker_kwds={"radius": 8})

In [ ]:
# explore nodes and edges together in a single map
nodes, edges = ox.graph_to_gdfs(G)
m = edges.explore(color="skyblue", tiles="cartodbdarkmatter")
nodes.explore(m=m, color="pink", marker_kwds={"radius": 6})

## Explore attributes of the nodes and edges visually

In [ ]:
# explore graph edges interactively, colored by length
edges.explore(tiles="cartodbdarkmatter", column="length", cmap="plasma")

In [ ]:
# explore graph nodes interactively, colored by betweenness centrality
nx.set_node_attributes(G, nx.betweenness_centrality(G, weight="length"), name="bc")
nodes = ox.graph_to_gdfs(G, edges=False)
nodes.explore(tiles="cartodbdarkmatter", column="bc", marker_kwds={"radius": 8})

## Explore routes as an interactive web map

In [ ]:
# explore a route interactively
route_edges = ox.routing.route_to_gdf(G, route, weight=weight)
route_edges.explore(tiles="cartodbpositron", style_kwds={"weight": 5})

In [ ]:
# explore graph edges and route together in a single map
m = edges.explore(color="maroon", tiles="cartodbdarkmatter")
m = route_edges.explore(m=m, color="yellow", style_kwds={"weight": 5})
m

In [ ]:
# or explore multiple routes together in a single map
routes = ox.k_shortest_paths(G, orig, dest, k=200, weight=weight)
gdfs = (ox.routing.route_to_gdf(G, route, weight=weight) for route in routes)
m = edges.explore(color="#222222", tiles="cartodbdarkmatter")
for route_edges in gdfs:
    m = route_edges.explore(m=m, color="cyan", style_kwds={"weight": 5, "opacity": 0.05})
m

## Explore building footprints, transit lines/stops, or amenities

In [ ]:
# define some options
place = "Pasadena, CA, USA"
tiles = "cartodbdarkmatter"
mk = {"radius": 6}

In [ ]:
# explore a city's bus stops and rail transit interactively
bus = ox.features_from_place(place, tags={"highway": "bus_stop"})
m = bus.explore(tiles=tiles, color="red", tooltip="name", marker_kwds=mk)
rail = ox.features_from_place(place, tags={"railway": "light_rail"})
m = rail.explore(m=m, tiles=tiles, color="yellow", tooltip="name")
stations = ox.features_from_place(place, tags={"railway": "station"})
stations.explore(m=m, tiles=tiles, color="yellow", tooltip="name", marker_kwds=mk)

In [ ]:
# explore a city's parks interactively
parks = ox.features_from_place(place, tags={"leisure": "park"})
parks.explore(tiles=tiles, color="lime", tooltip="name")

In [ ]:
# explore a neighborhood's buildings interactively
gdf = ox.features_from_place("SoHo, New York, NY", tags={"building": True})
cols = ["height", "addr:housenumber", "addr:street", "addr:postcode"]
gdf.explore(tiles="cartodbdarkmatter", tooltip=cols)

In [ ]:
# explore a neighborhood's buildings + street network interactively
place = "SoHo, New York, NY"
cols = ["height", "addr:housenumber", "addr:street", "addr:postcode"]
G = ox.graph_from_place(place, network_type="drive", truncate_by_edge=True)
gdf = ox.features_from_place(place, tags={"building": True})
m = gdf.explore(tiles=tiles, tooltip=cols)
ox.graph_to_gdfs(G, nodes=False).explore(m=m, color="yellow")

In [ ]:
# you can save a web map as an HTML file for reuse elsewhere
m.save("data/graph.html")